In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from collections import deque
import random
import time
import yaml

In [10]:
# TO DO:
# add gradient clipping?

# TO TEST:
# enforcing norm constraint on P
# how to do exploration
# whether this works at all?

# carries out mat*v on all v in batch_v
# mat = [n, m], batch_v = [batch_size, m], returns [batch_size, n]
def batch_matmul(mat, batch_v):
    return tf.transpose(tf.matmul(mat,tf.transpose(batch_v)))

def summarize_matrix(name, matrix):
    with tf.name_scope(name):
        eigvals, _ = tf.linalg.eigh(matrix)
        tf.summary.scalar('max_eig', tf.reduce_max(eigvals))
        tf.summary.scalar('min_eig', tf.reduce_min(eigvals))
        tf.summary.scalar('mean_eig', tf.reduce_mean(eigvals))

class klqr:
    # not currently doing value updates at varying rates
    # not currently doing double Q learning (what would this look like?)
    
    def __init__(self,config,sess):
        self.sess = sess
        
        self.x_dim = config['x_dim']
        self.z_dim = config['z_dim']
        self.a_dim = config['a_dim']
        self.lr = config['lr']
        self.horizon = config['horizon']
        self.gamma = config['discount_rate']

        
        ou_theta = config['ou_theta']
        ou_sigma = config['ou_sigma']
        self.config = config
        
        # Ornstein-Uhlenbeck noise for exploration -- code from Yuke Zhu
        self.noise_var = tf.Variable(tf.zeros([self.a_dim,1]))
        noise_random = tf.random_normal([self.a_dim,1], stddev=ou_sigma)
        self.noise = self.noise_var.assign_sub((ou_theta) * self.noise_var - noise_random)

        self.max_riccati_updates = config['max_riccati_updates']
        self.train_batch_size = config['train_batch_size']
        self.replay_buffer = ReplayBuffer(buffer_size=config['replay_buffer_size'])
        
        self.dynamics_weight = 1.0
        self.cost_weight = 1.0
        self.td_weight = 0.0
        
        self.experience_count = 0
        
        self.updates_so_far = 0
        
    def build_model(self):        

        with tf.variable_scope('model',reuse=tf.AUTO_REUSE):
            
            self.x_ = tf.placeholder(tf.float32,shape=[None, self.x_dim])
            self.xp_ = tf.placeholder(tf.float32,shape=[None, self.x_dim])
            self.a_ = tf.placeholder(tf.float32,shape=[None, self.a_dim])
            self.r_ = tf.placeholder(tf.float32,shape=[None])
            
            self.z = self.encoder(self.x_)
            self.zp = self.encoder(self.xp_)

            print('z shape:', self.z.get_shape())

            #init R

            self.R_asym = tf.get_variable('R_asym',shape=[self.a_dim,self.a_dim])
    #         self.R_asym = tf.Variable(np.random.rand(self.a_dim,self.a_dim) - 0.5)

            # working with Ra.T Ra so that inner product is norm(Rx) and not norm(R.T x)
            self.R = tf.matmul(tf.linalg.transpose(self.R_asym),self.R_asym)

            #init Q -- shape: z_dim * z_dim
            self.Q_asym = tf.get_variable('Q_asym',shape=[self.z_dim,self.z_dim])
            self.Q = tf.matmul(tf.linalg.transpose(self.Q_asym),self.Q_asym)

            #init P -- shape: z_dim * z_dim
            self.P = tf.get_variable('P',shape=[self.z_dim,self.z_dim],trainable=False,initializer=tf.initializers.identity)
            self.P_asym = tf.linalg.transpose(tf.cholesky(self.P))

            #init B -- shape: z_dim * u_dim
            self.B = tf.get_variable('B',shape=[self.z_dim,self.a_dim])
    #         self.B = tf.Variable(np.random.rand(self.z_dim,self.u_dim) - 0.5)

            #init A -- shape: z_dim * z_dim
            self.A = tf.get_variable('A',shape=[self.z_dim,self.z_dim])
    #         self.A = tf.Variable(np.random.rand(self.z_dim,self.z_dim) - 0.5)

            #define K -- shape: u_dim * z_dim
            #term1 = tf.matrix_inverse(self.R + tf.matmul(tf.matmul(tf.transpose(self.B),self.Q),self.B))
            term1 = tf.matrix_inverse(self.R + tf.linalg.transpose(self.B) @ self.P @ self.B)
            term2 = tf.linalg.transpose(self.B) @ self.P @ self.A
            self.K = -term1 @ term2
            self.policy_action = batch_matmul(self.K, self.z) # tf.transpose(tf.matmul(self.K,tf.transpose(self.z)))
            
            #make reward negative to convert to cost
            self.bootstrapped_value = -self.r_ + self.gamma*tf.square(tf.norm(batch_matmul(self.P_asym, self.zp), axis=1))

            action_cost = tf.square(tf.norm(batch_matmul(self.R_asym, self.a_), axis=1))#can simplify this by taking norm on other axis
            state_cost = tf.square(tf.norm(batch_matmul(self.Q_asym, self.z), axis=1)) 
            self.PABK = self.P_asym @ ( self.A + self.B @ self.K )
            Vzp = tf.square(tf.norm(batch_matmul(self.PABK, self.zp), axis=1))
            self.Qsa = action_cost + state_cost + Vzp
            
            # predict next state
            self.zp_pred = batch_matmul(self.A, self.z) + batch_matmul(self.B, self.a_)
            
            # predict observed reward
            self.r_pred = - action_cost - state_cost
            print(self.r_pred.get_shape())

            self.td_loss = tf.reduce_mean(tf.square(self.bootstrapped_value - self.Qsa))
            self.dynamics_loss = tf.reduce_mean(tf.square(self.zp - self.zp_pred))
            self.cost_pred_loss = tf.reduce_mean(tf.square(self.r_pred - self.r_))
            
            
            self.loss = self.td_weight*self.td_loss + self.dynamics_weight*self.dynamics_loss + self.cost_weight*self.cost_pred_loss
            global_step = tf.Variable(0, trainable=False, name='global_step')
            optimizer = tf.train.AdamOptimizer(self.lr)
            self.train_op = optimizer.minimize(self.loss, global_step=global_step)
            
            # utilities for doing riccati recursion
            self.reset_P_op = self.P.assign(self.Q)
            self.riccati_update_op = self.P.assign(self.riccati_recursion_step())
            
            # record summaries
            tf.summary.scalar('dynamics_loss', self.dynamics_loss)
            tf.summary.scalar('cost_pred_loss', self.cost_pred_loss)
            tf.summary.scalar('td_loss', self.td_loss)
            summarize_matrix('A', self.A)
#             summarize_matrix('B', self.B)
            summarize_matrix('Q', self.Q)
            summarize_matrix('R', self.R)
            summarize_matrix('P', self.P)
            
            self.merged = tf.summary.merge_all()
            self.train_writer = tf.summary.FileWriter('summaries')
            
            self.sess.run(tf.global_variables_initializer())

    
    def update_model(self):        
        #this function is mostly taken from Yuke's code
        if self.replay_buffer.count() < self.train_batch_size:
            return
        
        batch           = self.replay_buffer.getBatch(self.train_batch_size)
        
        states          = np.zeros((self.train_batch_size, self.x_dim))
        rewards         = np.zeros((self.train_batch_size))
        actions         = np.zeros((self.train_batch_size, self.a_dim))
        next_states     = np.zeros((self.train_batch_size, self.x_dim))

        for k, (s0, a, r, s1, done) in enumerate(batch):
            #currently throwing away done states; should fix this
            states[k] = s0
            rewards[k] = r
            actions[k] = a
            next_states[k] = s1
            # check terminal state
#             if not done:
#                 next_states[k] = s1
#                 next_state_mask[k] = 1

        summary, _ = self.sess.run([self.merged, self.train_op],
        {
        self.x_:  states,
        self.xp_: next_states,
        self.a_:  actions,
        self.r_:  rewards
        })
    
        self.train_writer.add_summary(summary, self.updates_so_far)
        self.updates_so_far += 1
    
        #possibly update target via Riccati recursion? or do standard target separation? 
    
    def riccati_recursion_step(self):
#         ABK = self.A + self.B @ self.K
#         APA = tf.transpose(ABK) @ self.P @ ABK 
#         return self.Q + tf.transpose(self.K) @ self.R @ self.K + self.gamma*APA
        return self.Q + tf.transpose(self.A) @ self.P @ self.A - tf.transpose(self.A) @ self.P @ self.B @ tf.matrix_inverse(self.R + tf.transpose(self.B) @ self.P @ self.B ) @ tf.transpose(self.P @ self.B) @ self.A
    
    def update_P(self):
#         print('updating P')
#         reset_q_op = 
#         self.P = tf.identity(self.Q)
#         for k in range(self.max_riccati_updates):
#             #do Riccati backup in tensorflow oh god why
#             ABK = self.A + tf.matmul(self.B,self.K)
#             APA = tf.matmul(tf.matmul(tf.transpose(ABK),self.P),ABK) #
#             self.P = self.Q + tf.matmul(tf.matmul(tf.transpose(self.K),self.R),self.K) + self.gamma*APA
        
#         self.P_asym = tf.transpose(tf.cholesky(self.P))
        sess.run(self.reset_P_op)
        for k in range(self.max_riccati_updates):
            sess.run(self.riccati_update_op)
        
        print('Q:',sess.run(self.Q))
        print('R:',sess.run(self.R))
        print('A:',sess.run(self.A))
        print('B:',sess.run(self.B))
        print('P:',sess.run(self.P))
            #TODO add a termination criterion for norm of Riccati update difference?
    
    def pi(self,x,explore=True):
        self.experience_count += 1
        x = np.reshape(x,(1,self.x_dim))
        a,w = self.sess.run([self.policy_action,self.noise], {self.x_: x})
        
        a = a + w.T if explore else a
        return a.tolist()[0]
        
    def store_experience(self,s,a,r,sp,done):
        # currently storing experience for every iteration
        self.replay_buffer.add(s, a, r, sp, done)
    
    def encoder(self,x,name="encoder",batch_norm=False):
#         layer_sizes = self.config['encoder_layers']
#         with tf.variable_scope(name,reuse=tf.AUTO_REUSE):
#             inp = x
#             for units in layer_sizes: 
#                 inp = tf.layers.dense(inputs=inp, units=units,activation=tf.nn.relu)

#             z = tf.layers.dense(inputs=inp, units=self.z_dim,activation=None)

#         if batch_norm:
#             z = tf.layers.batch_normalization(z)

        return x #z

class ReplayBuffer:
    # taken from Yuke Zhu's Q learning implementation
    
    def __init__(self, buffer_size):

        self.buffer_size = buffer_size
        self.num_experiences = 0
        self.buffer = deque()

    def getBatch(self, batch_size):
        # random draw N
        return random.sample(self.buffer, batch_size)

    def size(self):
        return self.buffer_size

    def add(self, state, action, reward, next_action, done):
        new_experience = (state, action, reward, next_action, done)
        if self.num_experiences < self.buffer_size:
          self.buffer.append(new_experience)
          self.num_experiences += 1
        else:
          self.buffer.popleft()
          self.buffer.append(new_experience)

    def count(self):
        # if buffer is full, return buffer size
        # otherwise, return experience counter
        return self.num_experiences

    def erase(self):
        self.buffer = deque()
        self.num_experiences = 0

In [11]:
# simulates the agent acting in env, yielding every N steps
# (decouples episode reseting mechanics from the training alg)
def experience_generator(agent, env, N):
    s = env.reset()
    n_steps = 0
    n_eps = 0
    last_cum_rew = 0
    cum_rew = 0
    while True:
        n_steps += 1
        a = agent.pi(s)
        print('s:',s)
        sp, r, done,_ = env.step(a)
        cum_rew += r
        if done:
            n_eps += 1
            last_cum_rew = cum_rew
            cum_rew = 0
            s = env.reset()
        else:
            agent.store_experience(s, a, r, sp, done)
            s = sp

        if n_steps % N == 0:
            yield (n_steps, n_eps, last_cum_rew)



def train_agent(agent, env,
                max_timesteps=0, max_episodes=0, max_iters=0, max_seconds=0, # time constraint
                n_transitions_between_updates=100,
                n_optim_steps_per_update=100,
                n_iters_per_p_update=1,
                ):

    # run an episode, and feed data to model
    episodes_so_far = 0
    timesteps_so_far = 0
    iters_so_far = 0
    tstart = time.time()

    assert sum([max_iters>0, max_timesteps>0, max_episodes>0, max_seconds>0])==1, "Only one time constraint permitted"

    exp_gen = experience_generator(agent, env, n_transitions_between_updates)

    while True:
        iters_so_far += 1
        if max_timesteps and timesteps_so_far >= max_timesteps:
            break
        elif max_episodes and episodes_so_far >= max_episodes:
            break
        elif max_iters and iters_so_far >= max_iters:
            break
        elif max_seconds and time.time() - tstart >= max_seconds:
            break

        print("********** Iteration %i ************"%iters_so_far)

        # gather experience
        timesteps_so_far, episodes_so_far, last_cum_rew = exp_gen.__next__()

        # optimize the model from collected data:
        for i in range(n_optim_steps_per_update):
            agent.update_model()

        if iters_so_far % n_iters_per_p_update == 0:
            agent.update_P()

        print("\tLast Episode Reward: %d"%last_cum_rew)
        # add other logging stuff here
        # add saving checkpoints here


In [12]:
with open('config.yml','r') as ymlfile:
    config = yaml.load(ymlfile)
    
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

/home/james/Dropbox/stanford/research/current/koopman-lqr/klqr_env/lib/python3.5/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [13]:
env = gym.make('LQ-v0')
agent = klqr(config,sess)
agent.build_model()
train_agent(agent,env,max_timesteps=100000)

z shape: (?, 3)
(?,)
********** Iteration 1 ************
s: [1.6992663128385295, -1.6860551689642693, -1.8588843079185695]
s: [1.9023328584428794, -1.0402942785997566, -1.3194429385429673]
s: [1.7987009461029553, -0.44738817418507837, -0.8518707088068507]
s: [1.4364776297116668, -0.3630877095333159, -0.920322548723233]
s: [1.2045400581810144, -0.2657068283532201, -0.8084310248198936]
s: [1.000171697963533, -0.12896833496276328, -0.6953615827199394]
s: [0.8051359538913856, -0.12276974583807841, -0.5767017780312107]
s: [0.7862555556726052, 0.08617628913070167, 0.031739051367438376]
s: [0.6814584745179795, 0.13329917604561092, 0.8351727104830494]
s: [0.47427802253131957, 0.2798497587166043, 1.6995325159507844]
s: [0.3944079459875197, 0.6413795964429495, 2.290597060295639]
s: [0.11482799065115878, 1.0943774688445642, 2.443272846914705]
s: [-0.15842176780924355, 1.3683870044012512, 2.4760446259831403]
s: [-0.4846013006121427, 1.5913319297467448, 2.4851417356612986]
s: [-1.1292660431186619, 

s: [748.2304737240374, 664.0911355682322, 424.6190063069393]
s: [941.6204359964077, 835.9054905919298, 534.5224402558598]
s: [1185.0241789256315, 1052.0661010851668, 672.6333056234082]
s: [1491.17801477, 1323.9241887608962, 846.5042242751567]
s: [1876.6530575245408, 1666.7167551403763, 1065.579553916219]
s: [2361.9891735503115, 2098.575145102949, 1341.2982804192027]
s: [2973.1322667463774, 2642.022416683717, 1688.243920121122]
s: [3742.33087361655, 3325.5272437257872, 2124.9643151490973]
s: [4710.565309590571, 4185.865482451922, 2674.597161908154]
s: [5929.21605471849, 5268.781811999206, 3366.7159739084664]
s: [7463.496497544172, 6632.022039138687, 4237.723545473102]
s: [9394.48588321365, 8347.741745494075, 5333.863901014055]
s: [11824.938329068811, 10507.492760491732, 6714.1216760917005]
s: [14884.093472034465, 13226.355514550527, 8450.920575458316]
s: [18734.812053951126, 16648.811122508825, 10637.304138137482]
s: [23581.983933957967, 20956.653653907102, 13389.32193578711]
s: [29683.

InvalidArgumentError: Got info = 2 for batch index 0, expected info = 0. Debug_info = heevd
	 [[Node: model/Q/SelfAdjointEigV2 = SelfAdjointEigV2[T=DT_FLOAT, compute_v=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](model/MatMul_1)]]
	 [[Node: model/Mean/_127 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_540_model/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'model/Q/SelfAdjointEigV2', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/james/Dropbox/stanford/research/current/koopman-lqr/klqr_env/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/hom